In [1]:
!pip install keras-cv==0.4.0 -q
!pip install -U tensorflow -q
!pip install tensorflow_addons
!pip install numba
!pip install matlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 595.3/595.3 kB 37.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 34.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for matlab: filename=matlab-0.1-py3-none-any.whl size=1154 sha256=a2a6e68e14e9977d329b692e65719ebfd9b89c5eef47d34e3472d64b86b9fdc6
  Stored in directory: /root/.cache/pip/wheels/d1/d3/76/5314058ee22e7957a18eec0291788462f1df178bb08223bdd2
Successfully built matlab


In [ ]:
from numba import cuda
device = cuda.get_current_device()
device.reset()

In [2]:
from textwrap import wrap
import os

import keras_cv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.experimental.numpy as tnp
from keras_cv.models.stable_diffusion.clip_tokenizer import SimpleTokenizer
from keras_cv.models.stable_diffusion.diffusion_model import DiffusionModel
from keras_cv.models.stable_diffusion.image_encoder import ImageEncoder
from keras_cv.models.stable_diffusion.noise_scheduler import NoiseScheduler
from keras_cv.models.stable_diffusion.text_encoder import TextEncoder
from tensorflow import keras

import torch, torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
import scipy.io
from scipy.io import loadmat
import torch.nn as nn
import torch.optim as optim
import time
import math
import random
from torchsummary import summary
from PIL import Image

import random

from google.colab import drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Set TensorFlow to use GPU device
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Mon Jun 19 06:23:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
weights_path = "/content/drive/MyDrive/FineTuningStableDiffusion/Weights/finetuned_stable_diffusion.h5"
img_height = img_width = 512
lymphosa_model = keras_cv.models.StableDiffusion(
    img_width=img_width, img_height=img_height
)
# We just reload the weights of the fine-tuned diffusion model.
lymphosa_model.diffusion_model.load_weights(weights_path)

By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE
3439090152/3439090152 [==============================] - 14s 0us/step


In [5]:
# Load the mat file
matPath = "/content/drive/MyDrive/FineTuningStableDiffusion/Data/DatasColor_29.mat"
data = scipy.io.loadmat(matPath)

DIV = data['DATA'][0][2]  # Division between training and test set
yE = data['DATA'][0][1]  # Labels of the patterns
Images = data["DATA"][0][0][0]  # Images

In [6]:
print(Images.shape)
print(Images[0].shape)

(374,)
(312, 417, 3)


In [7]:
# Generate images from prompts
prompts = ["microscopic image of chronic lymphocytic leukemia", "microscopic image of follicular lymphoma", "microscopic image of mantle cell lymphoma"]
images_to_generate = 3
outputs = {}

for prompt in prompts:
    generated_images = lymphosa_model.text_to_image(
        prompt, batch_size=images_to_generate, unconditional_guidance_scale=40
    )
    outputs.update({prompt: generated_images})

25/25 [==============================] - 58s 2s/step


In [123]:
# for prompt in outputs:
#   for i in range(len(outputs[prompt])):
#     image = outputs[prompt][i]
#     resized_image = image.copy()  # Create a copy of the image array
#     resized_image = Image.fromarray(resized_image).resize((417, 312))  # Resize the image
#     resized_image = np.array(resized_image)
#     # print(np.array(resized_image).shape)
print(data["DATA"][0][0][0].dtype)

In [8]:
# Directory to save the images
save_dir = "../data/Images"
# os.makedirs(save_dir, exist_ok=True)
resized_images = []

def append_images(images, title, save_dir):
    # os.makedirs(save_dir, exist_ok=True)
    for i in range(len(images)):
        image = images[i]
        # image_path = os.path.join(save_dir, f"{title}_{i}.png")
        # image.save(image_path)
        # Append the images to the existing images in the .mat file
        resized_image = image.copy()  # Create a copy of the image array
        resized_image = Image.fromarray(resized_image).resize((417, 312))  # Resize the image
        resized_image = np.array(resized_image)
        resized_images.append((resized_image))

labels = []
for prompt in outputs:
    # plot_images(outputs[prompt], prompt)
    append_images(outputs[prompt], prompt, save_dir)

    if prompt == prompts[0]:
        label = 1
    elif prompt == prompts[1]:
        label = 2
    elif prompt == prompts[2]:
        label = 3
    else:
        raise ValueError("Prompt not found")
    labels.extend([label] * images_to_generate)

In [9]:
# Shuffle the images and labels together
combined_data = list(zip(resized_images, labels))
random.shuffle(combined_data)
resized_images, labels = zip(*combined_data)

In [32]:
print(labels)

(3, 3, 2, 1, 1, 2, 3, 1, 2)


In [11]:
# Convert Images and labels to lists
resized_images = np.array(resized_images).tolist()
new_indices = np.arange(375, 375 + images_to_generate*3).tolist()
new_labels = np.array(labels).tolist()

yE_list = yE[0].tolist()
DIV_list = DIV[0].tolist()
Images_list = Images.tolist()

In [201]:
print(len(DIV_list))
# test = new_indices + DIV_list

374


In [12]:
# Create a new table with elements from index 299 to the end of Images_list
chopped_list = Images_list[:299]
Images_list = Images_list[299:]
new_images = chopped_list + resized_images + Images_list

chopped_list = DIV_list[:299]
DIV_list = DIV_list[299:]
new_div = chopped_list + new_indices + DIV_list

chopped_list = yE_list[:299]
yE_list = yE_list[299:]
new_labels = chopped_list + new_labels + yE_list

print(len(new_images))
print(len(new_div))
print(len(new_labels))

383
383
383


In [21]:
print(np.array(new_images).shape)

(383, 312, 417, 3)


In [18]:
data_dict_SD = {
    'images': np.array(new_images),
    'labels': new_labels,
    'index' : new_div
}

# Save as .mat file for method 1 and method 2.
#For method_1
file_path_SD = '/content/drive/MyDrive/FineTuningStableDiffusion/Data/artificially_augmented_DS_2.mat'
scipy.io.savemat(file_path_SD, data_dict_SD)

In [39]:
matPath = "/content/drive/MyDrive/FineTuningStableDiffusion/Data/artificially_augmented_DS_2.mat"
mat_data = scipy.io.loadmat(matPath)
test = mat_data['images'].astype(np.uint8)
print(((test[0].dtype)))
print(((mat_data['labels'][0][140])))
print((len(mat_data['index'][0])))
for i in range(len(mat_data['images'])):
  if mat_data['images'][i].shape != (312, 417,3) :
      print("pika")
      print(i)
      print(mat_data['images'][i].shape)

uint8
2
383
